## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
# File location and type
file_location = "/FileStore/tables/winequality_red-42ff5.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ";"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df).limit(10)

In [3]:
# Create a view or table

temp_table_name = "winequality_red"

df.createOrReplaceTempView(temp_table_name)

In [4]:
%sql

/* Query the created temp table in a SQL cell */

SELECT * FROM `winequality_red` LIMIT 10

In [5]:
# Replace whitespaces in column names with '_'

from pyspark.sql.functions import col

df_corr = df.select(col('fixed acidity').alias('fixed_acidity'), col('volatile acidity').alias('volatile_acidity'), \
                    col('citric acid').alias('citric_acid'), col('residual sugar').alias('residual_sugar'), col('chlorides'), \
                    col('free sulfur dioxide').alias('free_sulfur_dioxide'), col('total sulfur dioxide').alias('total_sulfur_dioxide'), \
                    col('density'), col('pH'), col('sulphates'), col('alcohol'), col('quality'))

display(df_corr).limit(10)

In [6]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "wine_quality_delta"

df_corr.write.format("delta").saveAsTable(permanent_table_name)

In [7]:
# Databases and global tables in Hive 

display(dbutils.fs.ls('user/hive/warehouse'))

In [8]:
%sql
-- Update the Delta table

UPDATE wine_quality_delta SET citric_acid = 0 WHERE citric_acid < 0.1

In [9]:
%sql
-- Check modified values 

SELECT * FROM wine_quality_delta LIMIT 10

In [10]:
%sql
-- Delete rows from the Delta table

DELETE FROM wine_quality_delta WHERE citric_acid = 0

In [11]:
%sql
-- Check new state of the table 

SELECT * FROM wine_quality_delta LIMIT 10

In [12]:
%sql
-- Look at the history of modifications

DESCRIBE HISTORY wine_quality_delta

In [13]:
%sql
-- Read version 0

SELECT * FROM wine_quality_delta VERSION AS OF 0 LIMIT 10


In [14]:
%sql
-- Read version 1 

SELECT * FROM wine_quality_delta TIMESTAMP AS OF '2020-04-09T15:31:28.000+0000' LIMIT 10